# WANN experiments on UCI dataset

In [1]:
import copy
import sys

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.constraints import MinMaxNorm

sys.path.append("../wann")
from utils import superconduct, domain, BaggingModels, cross_val
from uci_experiments import run_uci_experiments
from methods import *

from warnings import filterwarnings
filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Load Dataset

In [2]:
source = 0  # possible values: 0, 1, 2, 3
target = 2  # possible values: 0, 1, 2, 3

N = 10   # Number of labeled target data

data, X, y, cuts, split_col = superconduct()
shape = X.shape[1]

src_index = domain(data, cuts, split_col, source)
tgt_index = domain(data, cuts, split_col, target)

np.random.seed(0)
tgt_train_index, tgt_test_index = train_test_split(tgt_index, train_size=N)
train_index = np.concatenate((src_index, tgt_train_index))

std_sc = StandardScaler()
std_sc.fit(X[train_index])
X = std_sc.transform(X)
y = (y - y[train_index].mean()) / y[train_index].std()

## Base Estimator

In [3]:
def get_base_model(shape, activation=None, C=1, name="BaseModel"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(1, activation=activation,
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled, name=name)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def get_encoder(shape, C=1, name="encoder"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model

def get_task(shape, C=1, activation=None, name="task"):
    inputs = Input(shape=(shape,))
    modeled = Dense(1, activation=activation,
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model


base_estimator = BaggingModels(func=get_base_model,
                               n_models=1,
                               n_jobs=None,
                               shape=shape,
                               C=1,
                               random_state=0)
fit_params = dict(epochs=200,
                  batch_size=1000,
                  verbose=0)

## Examples

### Target only

In [4]:
tgt_only = copy.deepcopy(base_estimator)
tgt_only.fit(X[tgt_train_index], y[tgt_train_index], **fit_params)

y_pred = tgt_only.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.372


### Source only

In [5]:
src_only = copy.deepcopy(base_estimator)
src_only.fit(X[src_index], y[src_index], **fit_params)

y_pred = src_only.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.501


### No reweight

In [6]:
no_reweight = copy.deepcopy(base_estimator)
no_reweight.fit(X[train_index], y[train_index], **fit_params)

y_pred = no_reweight.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.453


### TrAdaBoostR2

In [4]:
tradaboost = TwoStageTrAdaBoostR2(func=get_base_model,
                                  n_jobs=None,
                                  verbose=1,
                                  C=1,
                                  random_state=0,
                                  shape=X.shape[1])
tradaboost.fit(X, y, [src_index, tgt_train_index], **fit_params)
y_pred = tradaboost.predict(X)
score= mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

cv error of estimator 0: 0.379 (0.3653948407)
cv error of estimator 1: 0.364 (0.3397635452)
cv error of estimator 2: 0.432 (0.5109063533)
cv error of estimator 3: 0.212 (0.2105985221)
cv error of estimator 4: 0.193 (0.2412913909)
cv error of estimator 5: 0.228 (0.2829834499)
cv error of estimator 6: 0.233 (0.2558910015)
cv error of estimator 7: 0.252 (0.2672290448)
Binary search's goal not meeted! Value is set to be the available best!
cv error of estimator 8: 0.249 (0.2281143615)
Binary search's goal not meeted! Value is set to be the available best!
cv error of estimator 9: 0.195 (0.1867582923)
Target score: 0.314


### DANN

In [8]:
dann = BaggingModels(DANN, n_models=1, n_jobs=None, random_state=0,
                     get_encoder=get_encoder, get_task=get_task, lambda_=0.05)

resize_tgt_ind = np.array([tgt_train_index[i%len(tgt_train_index)]
                           for i in range(len(src_index))])

dann.fit(X, y, index=[src_index, resize_tgt_ind, tgt_train_index], **fit_params)

y_pred = dann.predict(X)
score = mean_squared_error(y[tgt_test_index], y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.642


### WANN

In [9]:
wann = BaggingModels(WANN, n_models=1, n_jobs=None, random_state=0,
                     get_base_model=get_base_model, C=1, C_w=0.1)

wann.fit(X, y, index=[src_index, tgt_train_index], **fit_params)

y_pred = wann.predict(X)
score = mean_squared_error(y[tgt_test_index], 
                           y_pred[tgt_test_index])
print('Target score: %.3f'%score)

Target score: 0.298


#### Cross Validation Cw Selection

In [10]:
cross_val("WANN", X, y, src_index, None, tgt_train_index,
          params=[0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1],
          fit_params=fit_params, cv=5,
          get_base_model=get_base_model, C=1);

Cross Validation: param = 0.010 | score = 0.3518
Cross Validation: param = 0.020 | score = 0.3098
Cross Validation: param = 0.050 | score = 0.1774
Cross Validation: param = 0.100 | score = 0.1622
Cross Validation: param = 0.200 | score = 0.2018
Cross Validation: param = 0.500 | score = 0.2319
Cross Validation: param = 1.000 | score = 0.3209
Best: param = 0.100 | score = 0.1622


## Run Experiments

In [11]:
df = run_uci_experiments(method="WANN",
                         get_base_model=get_base_model,
                         get_encoder=get_encoder,
                         get_task=get_task,
                         C=1,
                         C_w=0.1,
                         lambda_=0.1,
                         sigma=0.1,
                         epochs=200,
                         batch_size=1000,
                         n_models=1,
                         n_jobs=None,
                         n_target_labeled=10,
                         random_state=0,
                         save=False)

Experiment for method: WANN


############# 0 #############
--------- 1 ----------
Target_score: 0.232
--------- 2 ----------
Target_score: 0.298
--------- 3 ----------
Target_score: 0.323
############# 1 #############
--------- 0 ----------
Target_score: 0.437
--------- 2 ----------
Target_score: 0.258
--------- 3 ----------
Target_score: 0.353
############# 2 #############
--------- 0 ----------
Target_score: 0.616
--------- 1 ----------
Target_score: 0.399
--------- 3 ----------
Target_score: 0.351
############# 3 #############
--------- 0 ----------
Target_score: 0.591
--------- 1 ----------
Target_score: 0.512
--------- 2 ----------
Target_score: 0.327


### Launch all experiments (all methods, 10 times)
Uncomment cell below to launch experiments

In [12]:
# %run -i ..\wann\uci_experiments